# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### SENZ'ALTRO BISTROT - Toscana

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187895-d15272936-Reviews-SENZ_ALTRO_Bistrot-Florence_Tuscany.html

Home --> Ristoranti --> Firenze --> Best_ranking--> SENZ_ALTRO_Bistrot

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187895-d15272936-Reviews-SENZ_ALTRO_Bistrot-Florence_Tuscany.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187895-d15272936-Reviews-or10-SENZ_ALTRO_Bistrot-Florence_Tuscany.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187895-d15272936-Reviews-or'
URL_p2 = '0-SENZ_ALTRO_Bistrot-Florence_Tuscany.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

100%|██████████| 27/27 [00:29<00:00,  1.29s/it]

Executed
total_pages:  28
Nomi:  278
Commenti:  278
Voti:  278
Scritto da:  278


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 278 
 Recensioni analizzabili: 194 
 Recensioni scartate: 84 



['RECENSIONE_TROPPO_LUNGA',
 'Abbiamo cenato in questo piccolo e grazioso ristorante per la sera del mio compleanno e siamo rimasti veramente soddisfatti! Piatti unici ed originali, accostamenti particolari e prezzi nella media per la qualità che il ristorante offre. \nServizio gentilissimo. \nConto ottimo! \nSenz’altro ci torneremo!!!',
 'Piatti buonissimi ed una grande qualità dei prodotti utilizzati il servizio è stato impeccabile e il personale molto cordiale',
 'Siamo rimasti colpiti dalla raffinatezza e dalla cucina di questo piccolo ma splendido ristorantino.\nMaterie prime di primissima qualità,per non parlare del personale qualificato e disponibile a spiegare ogni singolo piatto.\nAssolutamente consigliato .',
 'RECENSIONE_TROPPO_LUNGA',
 'Serata davvero speciale! piatti curati nei particolari, buonissimo il maialino,vini adatti ad ogni palato.Il servizio ottimo, gentilissimo il personale. Ritorneremo sicuramente!',
 'RECENSIONE_TROPPO_LUNGA',
 'Sono stata a cena con il mio ra

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'Cinzia85_10', 'salvopugliese', 'valevale8628', 'elisa_210578', 'Gianluca C', 'RougeVerlaine', 'jacopod691', '_Silvana58_G', 'kattivik75', 'Massimo M', 'S D', 'antonioaS1017LP', 'alfonsopS7972OJ', 'BirilloFirenze', 'Armando A', 'Tommaso C', 'Umberto V', 'Angelo_Galati', 'blueyes11prato', '227nicolaz', 'Massimo d', 'francescafiorilli810', 'Angela S', 'Erika P', 'PattyPatrizia', 'alessandra z', 'Sabrina P', 'Simona L', 'ilmatto94', 'giuseppecT8820YG', 'Antonio T', 'deniserapisardaz', 'gngiulianocentini', 'Rualb77', 'francescoN414', 'Z9570TCluigic', 'Francesca C', 'Alfio C', 'Melina L', 'Mariagrazia L', '122filippof', 'LoveTravel687476', 'giorgiof798', 'loresupermoto', 'Patrizia P', 'Giusi B', 'Carmen B', 'bloomblo', 'Cekkky', 'Andrea T', 'Ocram P', 'camaleonte2015', 'Salvatore G', 'veronica f', 'Gabriel B', 'eleonoram0203', 'Lisina89', 'francesco l', 'Gianluca P', 'Alex1966Juve'

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'male', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'female', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'male', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'female', 'female', 'female', 'male', 'unknown', 'female', 'mostly_female', 'unknown', 'unknown', 'female', 'mostly_female', 'female

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'F', 'F', 'F', 'M', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'F', 'F', 'F', 'F', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'M', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'M',

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  278
Commenti:  278
Voti:  278
Mobile:  278


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  91


[['Gianluca',
  'M',
  'Complimenti! Ormai non se ne può fare a meno...una cucina moderna e sensoriale. Qualcosa di particolarmente delizioso... Bravi giovani!',
  '5',
  'Mobile'],
 ['Massimo',
  'M',
  'Delizioso bistrot per le vie del centro. Piccolo ma molto accogliente. Piatti sublimi!!! Io e mia moglie ne siamo rimasti entusiasti. Ci ritorneremo sicuramente quando saremo a Firenze!',
  '5',
  'Mobile'],
 ['Armando',
  'M',
  'Ormai non possiamo farne a meno: piatti fusion di un sapore particolare e servizio da 10eLode. Consiglio vivamente questo bistrot...',
  '5',
  'Mobile'],
 ['Tommaso',
  'M',
  'Accolti caldamente da un personale giovanile con un ottimo aperitivo offerto; serviti con gentilezza e velocità. I prezzi sono proporzionati alla qualità ed alla preparazione che sta dietro agli ottimi piatti! Consigliato!',
  '5',
  'PC'],
 ['Umberto',
  'M',
  'Ristorante molto buono anche se il numero 1 a Firenze mi sembra esagerato. Ragazzi giovani, che hanno frequentato l’Alma e

### Creiamo la tabella

In [0]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_0.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_0.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,M,Gianluca,Complimenti! Ormai non se ne può fare a meno.....,Mobile,5
1,M,Massimo,Delizioso bistrot per le vie del centro. Picco...,Mobile,5
2,M,Armando,Ormai non possiamo farne a meno: piatti fusion...,Mobile,5
3,M,Tommaso,Accolti caldamente da un personale giovanile c...,PC,5
4,M,Umberto,Ristorante molto buono anche se il numero 1 a ...,PC,4
5,M,Massimo,"Giovani di successo, in giro per il mondo... P...",Mobile,5
6,F,Angela,"Bellissimo bistrot... pranzo ottimo, maialino ...",Mobile,5
7,M,Antonio,Mai mangiato così bene in vacanza.... Bistrot ...,Mobile,5
8,F,Francesca,Eccelente ristorante con cuochi molto esperti ...,Mobile,5
9,M,Alfio,"Ero già stato qui in precedenza, così domenica...",Mobile,5
